# Get started!

In [ ]:
from pathlib import Path
import polars as pl
from ebrec.utils._constants import *
from ebrec.utils._python import compute_npratio, create_lookup_dict

In [ ]:
%pip install matplotlib seaborn
%pip install pyarrow
%pip install pandas
%pip install xlsx2csv

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.ticker as mticker
from matplotlib.ticker import MaxNLocator

## Help functions

In [ ]:
EB_COLOR = "#bd1118"


def save_figure(fig, save_path: str = None) -> None:
    if save_path is not None:
        path = Path(save_path)
        path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(path, dpi=300)


def plot_histogram(
    df: pl.DataFrame,
    column_name: str = None,
    stat: str = "density",
    save_path: str = None,
    x_max: int = None,
    y_max: int = None,
    binwidth: int = None,
    num_xticks: int = None,
    num_yticks: int = None,
    fontsize: int = 12,
    bins: int = "auto",
) -> None:
    # =>
    fig, ax = plt.subplots(figsize=(18, 14))
    sns.set_theme(style="whitegrid", font_scale=fontsize / 12)
    sns.histplot(
        data=df,
        x=column_name,
        color=EB_COLOR,
        binwidth=binwidth,
        alpha=0.5,
        stat=stat,
        bins=bins,
    )
    if x_max is not None:
        ax.set_xlim([0, x_max])
    if y_max is not None:
        ax.set_ylim([0, y_max])
    ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=True))
    ax.xaxis.set_major_locator(MaxNLocator(integer=True, nbins=num_xticks))
    if num_yticks is not None:
        ax.yaxis.set_major_locator(MaxNLocator(nbins=num_yticks, prune="lower"))

    plt.grid(axis="x")
    plt.title("")
    plt.xlabel("")
    plt.ylabel(stat.capitalize())
    plt.ticklabel_format(style="sci", axis="y", scilimits=(0, 0))
    plt.tight_layout()
    save_figure(fig, save_path=save_path)
    plt.show()


def create_bar_plot(
    df: pd.DataFrame,
    x_col: str,
    y_col: str,
    xlabel: str = "",
    ylabel: str = "",
    title: str = None,
    rotation: int = 0,
    fontsize: int = 12,
    colors: list[str] = None,
    y_as_percentage: bool = False,
    y_max: float = None,
    save_path: str = None,
):
    # Set style
    sns.set_theme(style="whitegrid", font_scale=fontsize / 12)
    # Create bar plot
    fig, ax = plt.subplots(figsize=(18, 14))
    ax = sns.barplot(
        data=df,
        x=x_col,
        y=y_col,
        alpha=0.80,
        palette=colors,
        legend=False,
        hue=x_col,
    )
    if y_as_percentage:
        # Format y-axis labels as percentage
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))
    if y_max is not None:
        ax.set_ylim([0, y_max])
    if title:
        plt.title(title)

    # Set labels and title
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=rotation)

    # Remove top and right borders
    # sns.despine()
    plt.tight_layout()
    save_figure(fig, save_path=save_path)
    plt.show()


def create_plot(
    x,
    xlabel: str = "",
    ylabel: str = "",
    title: str = None,
    rotation: int = 0,
    marker: str = "",
    linestyle: str = "-",
    fontsize: int = 12,
    markersize: float = 2.0,
    linewidth: float = 2.0,
    color: str = None,
    y_as_percentage: bool = False,
    y_max: float = None,
    x_max: float = None,
    num_xticks: int = None,
    num_yticks: int = None,
    save_path: str = None,
):
    fig, ax = plt.subplots(figsize=(18, 14))
    plt.plot(
        x,
        linewidth=linewidth,
        color=color,
        marker=marker,
        markersize=markersize,
        linestyle=linestyle,
    )
    if y_as_percentage:
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))
    else:
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=True))
    if y_max is not None:
        ax.set_ylim([0, y_max])
    if x_max is not None:
        ax.set_xlim([0, x_max])
    if title:
        ax.set_title(title, fontsize=fontsize)

    ax.xaxis.set_major_locator(MaxNLocator(integer=True, nbins=num_xticks))
    if num_yticks is not None:
        ax.yaxis.set_major_locator(MaxNLocator(nbins=num_yticks, prune="lower"))

    ax.set_xlabel(xlabel, fontsize=fontsize)
    ax.set_ylabel(ylabel, fontsize=fontsize)
    ax.tick_params(axis="x", labelsize=fontsize)
    ax.tick_params(axis="y", labelsize=fontsize)
    plt.xticks(rotation=rotation)
    plt.tight_layout()
    save_figure(fig, save_path=save_path)
    plt.show()


def add_word_count_column(
    df: pl.DataFrame, column: str, column_alias: str
) -> pl.DataFrame:
    return df.with_columns(
        pl.when(pl.col(column) != "")
        .then(pl.col(column).str.split(by=" ").list.len())
        .otherwise(0)
        .alias(column_alias)
    )

----
# Load dataset

In [ ]:
PATH = Path("~/Git Repositories/ebnerd-benchmark/data")
TRAIN_VAL_SPLIT = f"ebnerd_large"  # [ebnerd_demo, ebnerd_small, ebnerd_large]
TEST_SPLIT = f"ebnerd_testset"  # "ebnerd_testset", "ebnerd_testset_gt"

df_behaviors_train = df_behaviors = pl.scan_parquet(
    PATH.joinpath(TRAIN_VAL_SPLIT, "train", "behaviors.parquet")
)
df_history_train = df_behaviors = pl.scan_parquet(
    PATH.joinpath(TRAIN_VAL_SPLIT, "train", "history.parquet")
)
df_behaviors_val = df_behaviors = pl.scan_parquet(
    PATH.joinpath(TRAIN_VAL_SPLIT, "validation", "behaviors.parquet")
)
df_history_val = df_behaviors = pl.scan_parquet(
    PATH.joinpath(TRAIN_VAL_SPLIT, "validation", "history.parquet")
)
df_behaviors_test = df_behaviors = (
    pl.scan_parquet(PATH.joinpath(TEST_SPLIT, "test", "behaviors.parquet"))
    .filter(~pl.col(DEFAULT_IS_BEYOND_ACCURACY_COL))
    .drop(DEFAULT_IS_BEYOND_ACCURACY_COL)
)
df_history_test = df_behaviors = pl.scan_parquet(
    PATH.joinpath(TEST_SPLIT, "test", "history.parquet")
)
df_behaviors_test_ba = df_behaviors = pl.scan_parquet(
    PATH.joinpath(TEST_SPLIT, "test", "behaviors.parquet")
).filter(pl.col(DEFAULT_IS_BEYOND_ACCURACY_COL))
df_articles = pl.scan_parquet(PATH.joinpath("articles.parquet")).collect()

PLOT_PATH = Path("plot")

## Define some help names

In [ ]:
N_INVIEW_ARTICLES = "inview_len"
N_WORDS_TITLE = "title_len"
N_WORDS_SUBTITLE = "subtitle_len"
N_WORDS_BODY = "body_len"
CATEGORY_DIST_NAME = "category_distribution"
FONTSIZE = 90

## Concat datasets

In [ ]:
if DEFAULT_CLICKED_ARTICLES_COL not in df_behaviors_test.columns:
    df_behaviors_test = df_behaviors_test.with_columns(
        pl.Series(DEFAULT_CLICKED_ARTICLES_COL, [[0]], dtype=pl.List(pl.Int32))
    )

df_behaviors = pl.concat(
    [df_behaviors_train, df_behaviors_val, df_behaviors_test]
).with_columns(pl.col(DEFAULT_INVIEW_ARTICLES_COL).list.len().alias(N_INVIEW_ARTICLES))

df_history = pl.concat([df_history_train, df_history_val, df_history_test])

In [ ]:
TIME_HIST_COL = "impression_time_fixed"
df_history_unique_hist_interactions = (
    df_history.select(DEFAULT_USER_COL, pl.col(TIME_HIST_COL))
    .explode(TIME_HIST_COL)
    .group_by(DEFAULT_USER_COL)
    .agg(pl.col(TIME_HIST_COL).unique())
    .collect()
)

In [ ]:
df_articles = (
    df_articles.pipe(add_word_count_column, column="title", column_alias=N_WORDS_TITLE)
    .pipe(add_word_count_column, column="subtitle", column_alias=N_WORDS_SUBTITLE)
    .pipe(add_word_count_column, column="body", column_alias=N_WORDS_BODY)
)

# Dataset overview:

## Behaviors Train/Val:

In [ ]:
print(df_behaviors_train.columns)
df_behaviors_train.head(2).collect()

In [ ]:
df_history.head(2).collect()

## Articles:

In [ ]:
print(df_articles.columns)
df_articles.head(2)

## More descriptive:

In [ ]:
n_pos = (
    df_behaviors.select(pl.col(DEFAULT_CLICKED_ARTICLES_COL).list.len()).sum().collect()
)[DEFAULT_CLICKED_ARTICLES_COL][0]
n_neg = (
    df_behaviors.select(
        pl.col(DEFAULT_INVIEW_ARTICLES_COL).list.len()
        - pl.col(DEFAULT_CLICKED_ARTICLES_COL).list.len()
    )
    .sum()
    .collect()
)[DEFAULT_INVIEW_ARTICLES_COL][0]

n_samples_in_history = df_history_unique_hist_interactions.select(
    pl.col(TIME_HIST_COL).list.len()
).sum()[TIME_HIST_COL][0]

n_impressions = df_behaviors.select(DEFAULT_USER_COL).collect().shape[0]

n_users = df_behaviors.select(DEFAULT_USER_COL).unique().collect().shape[0]
n_sso_users = (
    df_behaviors.select(DEFAULT_USER_COL, DEFAULT_IS_SSO_USER_COL)
    .unique()
    .select(pl.col(DEFAULT_IS_SSO_USER_COL))
    .sum()
    .collect()[DEFAULT_IS_SSO_USER_COL][0]
)
n_subscriber_users = (
    df_behaviors.select(DEFAULT_USER_COL, DEFAULT_IS_SUBSCRIBER_COL)
    .unique()
    .sum()
    .collect()[DEFAULT_IS_SUBSCRIBER_COL][0]
)
n_articles = df_articles.select(DEFAULT_ARTICLE_ID_COL).unique().shape[0]
n_categories = df_articles.select(DEFAULT_CATEGORY_COL).unique().shape[0]
n_subcategories = (
    df_articles.select(pl.col(DEFAULT_SUBCATEGORY_COL).explode()).unique().shape[0]
)

descriptive_dict = {
    f"# News": n_articles,
    f"# Users (unique)": n_users,
    f"# News categories": n_categories,
    f"# Impressions": n_impressions,
    f"# History Interactions": n_samples_in_history,
    f"# Total dataset interactions (Impressions + History)": n_impressions
    + n_samples_in_history,
    f"# News subcategories": n_subcategories,
    f"# Positive": n_pos,
    f"# Negative": n_neg,
    f"NP-ratio": round(compute_npratio(n_pos=n_pos, n_neg=n_neg), 2),
    f"Avg. impression per user": round(n_impressions / n_users, 2),
    f"Avg. title len. (words)": round(
        df_articles.select(pl.col(N_WORDS_TITLE)).mean()[N_WORDS_TITLE][0], 2
    ),
    f"Std. title len. (words)": round(
        df_articles.select(pl.col(N_WORDS_TITLE)).std()[N_WORDS_TITLE][0], 2
    ),
    f"Avg. abstract len. (words)": round(
        df_articles.select(pl.col(N_WORDS_SUBTITLE)).mean()[N_WORDS_SUBTITLE][0],
        2,
    ),
    f"Std. abstract len. (words)": round(
        df_articles.select(pl.col(N_WORDS_SUBTITLE)).std()[N_WORDS_SUBTITLE][0],
        2,
    ),
    f"Avg. body len. (words)": round(
        df_articles.select(pl.col(N_WORDS_BODY)).mean()[N_WORDS_BODY][0],
        2,
    ),
    f"Std. body len. (words)": round(
        df_articles.select(pl.col(N_WORDS_BODY)).std()[N_WORDS_BODY][0],
        2,
    ),
    f"# SSO users": n_sso_users,
    f"# Subscriber users": n_subscriber_users,
}
_ = [print(f"{key}: {value}") for key, value in descriptive_dict.items()]

In [ ]:
def compute_overlap(df1, df2):
    df_concat = pl.concat([df1, df2]).unique().shape[0]
    return df1.shape[0] / df_concat


users_train = df_behaviors_train.select(pl.col(DEFAULT_USER_COL).unique()).collect()
users_valid = df_behaviors_val.select(pl.col(DEFAULT_USER_COL).unique()).collect()
users_test = df_behaviors_test.select(pl.col(DEFAULT_USER_COL).unique()).collect()
train_val, train_test, val_test, train_ValTest, val_TrainTest, test_TrainVal = (
    compute_overlap(users_train, users_valid),
    compute_overlap(users_train, users_test),
    compute_overlap(users_valid, users_test),
    compute_overlap(users_train, pl.concat([users_valid, users_test]).unique()),
    compute_overlap(users_valid, pl.concat([users_train, users_test]).unique()),
    compute_overlap(users_test, pl.concat([users_train, users_valid]).unique()),
)
print(f"Train-Val overlap: {round(train_val*100,2)}%")
print(f"Train-Test overlap: {round(train_test*100, 2)}%")
print(f"Val-Test overlap: {round(val_test*100, 2)}%")
print(f"Train-(Val+Test) overlap: {round(train_ValTest*100, 2)}%")
print(f"Val-(Train+Test) overlap: {round(val_TrainTest*100, 2)}%")
print(f"Test-(Train+Val) overlap: {round(test_TrainVal*100, 2)}%")

In [ ]:
gender_total = (
    df_behaviors.select(DEFAULT_USER_COL, DEFAULT_GENDER_COL)
    .drop_nulls()
    .group_by(DEFAULT_USER_COL)
    .agg(pl.col(DEFAULT_GENDER_COL).first())
    .collect()
)
m_total = gender_total.filter(pl.col(DEFAULT_GENDER_COL) == 0).shape[0]
w_total = gender_total.filter(pl.col(DEFAULT_GENDER_COL) == 1).shape[0]
m_w_total = m_total + w_total
print(
    f"Users with 'gender' in total dataset: {m_w_total}, where {m_total} are men ({round(m_total/m_w_total*100, 2)}%) and {w_total} are women ({round(w_total/m_w_total*100, 2)}%)"
)
gender_ba = (
    df_behaviors_test_ba.select(
        DEFAULT_USER_COL, DEFAULT_GENDER_COL, DEFAULT_IS_BEYOND_ACCURACY_COL
    )
    .drop_nulls()
    .group_by(DEFAULT_USER_COL)
    .agg(pl.col(DEFAULT_GENDER_COL).first())
    .collect()
)
m_ba = gender_ba.filter(pl.col(DEFAULT_GENDER_COL) == 0).shape[0]
w_ba = gender_ba.filter(pl.col(DEFAULT_GENDER_COL) == 1).shape[0]
m_w_ba = m_ba + w_ba
print(
    f"Users with 'gender' in beyond-accuracy dataset: {m_w_ba}, where {m_ba} are men ({round(m_ba/m_w_ba*100, 2)}%) and {w_ba} are women ({round(w_ba/m_w_ba*100, 2)}%)"
)

-----
# Visualizations:

## Textual features in Articles:

### Title

In [ ]:
#
print(df_articles[N_WORDS_TITLE].describe())
plot_histogram(
    df_articles.filter(pl.col(N_WORDS_TITLE) <= 18).select(N_WORDS_TITLE),
    num_xticks=4,
    num_yticks=3,
    fontsize=FONTSIZE,
    binwidth=1,
    column_name=N_WORDS_TITLE,
    save_path=PLOT_PATH.joinpath(N_WORDS_TITLE + ".png"),
)

### Subtitle

In [ ]:
print(df_articles[N_WORDS_SUBTITLE].describe())
plot_histogram(
    df_articles.filter(pl.col(N_WORDS_SUBTITLE) <= 50).select(N_WORDS_SUBTITLE),
    num_xticks=4,
    num_yticks=3,
    fontsize=FONTSIZE,
    binwidth=1,
    column_name=N_WORDS_SUBTITLE,
    save_path=PLOT_PATH.joinpath(N_WORDS_SUBTITLE + ".png"),
)

### Body

In [ ]:
print(df_articles[N_WORDS_BODY].describe())
plot_histogram(
    df_articles.filter(pl.col(N_WORDS_BODY) <= 1200).select(N_WORDS_BODY),
    num_xticks=4,
    num_yticks=3,
    fontsize=FONTSIZE,
    bins=70,
    binwidth=None,
    column_name=N_WORDS_BODY,
    save_path=PLOT_PATH.joinpath(N_WORDS_BODY + ".png"),
)

## Number of in-view articles per impression

In [ ]:
print(df_behaviors.select(N_INVIEW_ARTICLES).describe())
# =>
plot_histogram(
    df_behaviors.select(N_INVIEW_ARTICLES).collect(),
    x_max=30,
    num_xticks=4,
    num_yticks=3,
    fontsize=90,
    binwidth=1,
    column_name=N_INVIEW_ARTICLES,
    save_path=PLOT_PATH.joinpath(N_INVIEW_ARTICLES + f".png"),
)

## Front vs. Article page:

In [ ]:
df_frontpage = (
    pl.concat([df_behaviors_train, df_behaviors_val])
    .select(DEFAULT_ARTICLE_ID_COL, DEFAULT_READ_TIME_COL)
    .filter(pl.col(DEFAULT_ARTICLE_ID_COL).is_null())
    .collect()
)
df_articlepage = (
    pl.concat([df_behaviors_train, df_behaviors_val])
    .select(DEFAULT_ARTICLE_ID_COL, DEFAULT_READ_TIME_COL)
    .filter(pl.col(DEFAULT_ARTICLE_ID_COL).is_not_null())
    .collect()
)
total_ = df_frontpage.shape[0] + df_articlepage.shape[0]

### Inviews - front- vs. article page:

In [ ]:
article_page_frac = round(df_articlepage.shape[0] / total_, 2)
frontpage_frac = round(1 - article_page_frac, 2)
#
sizes = [frontpage_frac, article_page_frac]
labels = f"Frontpage ({frontpage_frac})", f"ArticlePage ({article_page_frac})"
fig, ax = plt.subplots()
front_article_page = ax.pie(sizes, labels=labels, colors=["blue", "orange"])
plt.tight_layout()
save_figure(fig, save_path=PLOT_PATH.joinpath("front_article_page.png"))
plt.show()

### Readtime - front- vs. article page:

In [ ]:
N_SHOW = 120
plot_histogram(
    df_frontpage.filter(pl.col(DEFAULT_READ_TIME_COL) < N_SHOW).select(
        DEFAULT_READ_TIME_COL
    ),
    column_name=DEFAULT_READ_TIME_COL,
    x_max=N_SHOW,
    bins=60,
    num_xticks=6,
    fontsize=90,
    save_path=PLOT_PATH.joinpath("front_read_time.png"),
)

In [ ]:
N_SHOW = 240
plot_histogram(
    df_articlepage.filter(pl.col(DEFAULT_READ_TIME_COL) < N_SHOW).select(
        DEFAULT_READ_TIME_COL
    ),
    column_name=DEFAULT_READ_TIME_COL,
    x_max=N_SHOW,
    bins=60,
    num_xticks=6,
    fontsize=90,
    save_path=PLOT_PATH.joinpath("article_read_time.png"),
)

## Article category distributions:

In [ ]:
df_articles[DEFAULT_CATEGORY_STR_COL].value_counts(sort=True).head(15)

In [ ]:
translate = {
    "nyheder": "NWS",  # news
    #
    "underholdning": "ENT",  # entertainment
    "musik": "ENT",  # entertainment
    #
    "krimi": "CRM",  # crime
    #
    "sport": "SPT",  # sports
    #
    "forbrug": "LFS",  # Lifestyle
    "biler": "LFS",  # Lifestyle
    "ferie": "LFS",  # Lifestyle
    "vin": "LFS",  # Lifestyle
    #
    "sex_og_samliv": "SRL",  # S&R
    #
    "nationen": "OPN",  # Opinion
    "opinionen": "OPN",  # Opinion
}
df_category = (
    df_articles[DEFAULT_CATEGORY_STR_COL]
    .replace(translate, default="MSC")
    .value_counts(sort=True)
    .with_columns(pl.col("count") / pl.col("count").sum())
)
df_category.with_columns(pl.col("count").cum_sum().name.suffix("_cum_sum"))

In [ ]:
create_bar_plot(
    df_category.to_pandas(),
    x_col=DEFAULT_CATEGORY_STR_COL,
    y_col="count",
    y_as_percentage=True,
    colors=sns.color_palette(n_colors=df_category.shape[0]),
    fontsize=75,
    rotation=45,
    save_path=PLOT_PATH.joinpath(CATEGORY_DIST_NAME + ".png"),
)

## Candidate-list distribution

In [ ]:
ba_aids = (
    df_behaviors_test_ba.filter(pl.col(DEFAULT_IS_BEYOND_ACCURACY_COL))
    .select(pl.col(DEFAULT_INVIEW_ARTICLES_COL).first().explode())
    .collect()
)[DEFAULT_INVIEW_ARTICLES_COL].to_list()

In [ ]:
df_category_ba = (
    df_articles.filter(pl.col(DEFAULT_ARTICLE_ID_COL).is_in(ba_aids))[
        DEFAULT_CATEGORY_STR_COL
    ]
    .replace(translate, default="MSC")
    .value_counts(sort=True)
    .with_columns(pl.col("count") / pl.col("count").sum())
)
df_category_ba.with_columns(pl.col("count").cum_sum().name.suffix("_cum_sum"))

In [ ]:
create_bar_plot(
    df_category_ba.to_pandas(),
    x_col=DEFAULT_CATEGORY_STR_COL,
    y_col="count",
    y_as_percentage=True,
    colors=sns.color_palette(n_colors=df_category.shape[0]),
    fontsize=75,
    rotation=45,
    save_path=PLOT_PATH.joinpath(CATEGORY_DIST_NAME + "_ba" + ".png"),
)

# DONE 🚀